Владимир Кирилкин ПИ18-2

ML 1.5. Классификация текстов
https://docs.google.com/document/d/1zE5qsLRtLK9UgESbWeFUnoXwut8iaL2yMM8WZ_4HE0g/edit

In [1]:
import time
import numpy as np
from scipy.stats import sem
from sklearn import metrics, svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from json import load
with open("data/stopwords.json") as file:
    stopwords = load(file)

Достаём обучающую и тестовую выборки, оставив только текст

In [ ]:
train = fetch_20newsgroups(subset="train",remove=("headers", "footers", "quotes"))
test = fetch_20newsgroups(subset="test", remove=("headers", "footers", "quotes"))
all_set = fetch_20newsgroups(subset="all", remove=("headers", "footers", "quotes"))

In [3]:
train = fetch_20newsgroups(subset="train")
test = fetch_20newsgroups(subset="test")
all_set = fetch_20newsgroups(subset="all")

In [4]:
clf = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stopwords,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('clf', MultinomialNB(alpha=0.01)),
])

In [5]:
clf.fit(train.data, train.target)


Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             'your', 'yours', 'yourself',
                                             'yourselves', 'he', 'him', 'his',
                                             'himself', 'she', 'her', 'hers',
                                             'herself', 'it', 'its', 'itself',
                                             'they', 'them', 'their', 'theirs',
                                             'themselves', 'what', ...],
                                 token_pattern='\\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\\b')),
                ('clf', MultinomialNB(alpha=0.01))])

In [10]:
print(train.data[2])

From: twillis@ec.ecn.purdue.edu (Thomas E Willis)
Subject: PB questions...
Organization: Purdue University Engineering Computer Network
Distribution: usa
Lines: 36

well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk

Метрики полученной модели

In [6]:
predict = clf.predict(test.data)
print(f"""
accuracy: {metrics.accuracy_score(test.target, predict)}
precision: {metrics.precision_score(test.target, predict, average='weighted', zero_division=0)}
recall: {metrics.recall_score(test.target, predict, average='weighted', zero_division=0)}""")



accuracy: 0.8466542750929368
precision: 0.8479552575009973
recall: 0.8466542750929368


In [7]:
cv = KFold(5, shuffle=True, random_state=0).get_n_splits(len(train.target))
scores = cross_val_score(clf, all_set.data, all_set.target, cv=cv)
print(f"Mean: {np.mean(scores)} ±{sem(scores)}")

Mean: 0.922582874532079 ±0.001959219474579009


Пробуем другие классификаторы

In [51]:
models = [
    LogisticRegression(max_iter=10000, n_jobs=-1),
    svm.SVC(kernel='linear'),
    svm.SVC(kernel='rbf'),
    KNeighborsClassifier(n_jobs=-1),
    MLPClassifier(max_iter=10000)
]

for model in models:
    start_time = time.time()
    clf = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stopwords,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('clf', model),
])
    clf.fit(train.data, train.target)

    spend_time = time.time() - start_time
    predict = clf.predict(test.data)
    print(
        clf,
        "\nЗатрачено секунд ",
        spend_time,
        "\nScore",
        clf.score(test.data, test.target),
        "\nМетрика достоверности предсказания",
        metrics.accuracy_score(test.target, predict),
        "\nМетрика точности",
        metrics.precision_score(test.target, predict, average="weighted", zero_division=0),
        "\nМетрика полноты",
        metrics.recall_score(test.target, predict,average="weighted", zero_division=0),
        "\n"
    )

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             'your', 'yours', 'yourself',
                                             'yourselves', 'he', 'him', 'his',
                                             'himself', 'she', 'her', 'hers',
                                             'herself', 'it', 'its', 'itself',
                                             'they', 'them', 'their', 'theirs',
                                             'themselves', 'what', ...],
                                 token_pattern='\\b[a-z0-9_\\-\\.]+[a-z][a-z0-9_\\-\\.]+\\b')),
                ('clf', LogisticRegression(max_iter=10000, n_jobs=-1))]) 
Затрачено секунд  75.88197684288025 
Score 0.679368029739777 
Метрика достоверности предсказания 0.679368029739777 
Метрика точности 0.689541027160054 
Метрика полноты 0.67936802973977

Лучше всего тут LogisticRegression, он же оказался самым быстрым

KNeighbors чтото вообще не смог обучиться

MLPClassifier хоть и оказался самым точным, но работал больше 45 минут... Это слишком долго